In [1]:
import pandas as pd
import numpy as np
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

#### The data

In [2]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [3]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [4]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [5]:
def ffnn(X_train, y_train, epochs=500, optimizer='RMSProp', dropout=0.2):
    model = tf.keras.Sequential() 
    model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
    model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 

    model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
                  loss=tf.keras.losses.MeanSquaredError(),
                 )
    model.fit(X_train, 
          y_train,
          batch_size=32,
          epochs=epochs, # default=1, Number of epochs to train the model
          callbacks=None,
          validation_split=0.2, 
         )
    return model

In [6]:
def scale_data(X):
    scaler = MinMaxScaler()
    X_scaled = X.copy()
    X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
    return X_scaled

#### delete from here

In [7]:
# idx = 0
# device_indices = final_df[final_df.latitude==latitudes[idx]].index
# device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
# assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
# test_df = final_df.loc[device_indices]
# test_df.head()

In [8]:
# train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
# assert(len(train_df.longitude.unique()) == len(longitudes)-1)
# assert len(final_df) == len(test_df) + len(train_df)
# train_df.head()

In [9]:
# X_train = train_df.iloc[:, 0:-1]
# y_train = train_df.iloc[:, -1]
# X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)

# X_test = test_df.iloc[:, 0:-1]
# y_test = test_df.iloc[:, -1]
# X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [10]:
# X_train[0]

In [11]:
# X_train_scaled = scale_data(X_train)
# X_train_scaled

In [12]:
# model = ffnn(X_train, y_train, 500, 'RMSProp')

In [13]:
# y_pred = model.predict(X_test) 
# y_pred.shape, y_test.reshape(-1, 1).shape

In [14]:
# np.min(y_pred), np.mean(y_pred), np.max(y_pred)

In [15]:
# mean_squared_error(y_test, y_pred)

In [16]:
# y_train_pred = model.predict(X_train)
# mean_squared_error(y_train, y_train_pred)

#### end here

In [17]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    X_test_scaled = scale_data(X_test)
    
    model = ffnn(X_train_scaled, y_train, optimizer='Adam', dropout=0.8)
    y_pred = model.predict(X_test_scaled) 
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [18]:
rmse_list = []
for i in range(len(latitudes)):
    rmse = cross_validation(final_df, i)
    rmse_list.append(rmse)
    print(f'{device_ids[i]} successful')

Epoch 1/500
294/294 [==============================] - 1s 3ms/step - loss: 1253.9276 - val_loss: 1512.2953
Epoch 2/500
294/294 [==============================] - 1s 2ms/step - loss: 695.7789 - val_loss: 1463.4381
Epoch 3/500
294/294 [==============================] - 1s 2ms/step - loss: 617.4050 - val_loss: 1461.6874
Epoch 4/500
294/294 [==============================] - 1s 2ms/step - loss: 580.5906 - val_loss: 1427.0968
Epoch 5/500
294/294 [==============================] - 1s 2ms/step - loss: 568.1932 - val_loss: 1431.9536
Epoch 6/500
294/294 [==============================] - 1s 2ms/step - loss: 554.2771 - val_loss: 1369.4955
Epoch 7/500
294/294 [==============================] - 1s 2ms/step - loss: 549.0443 - val_loss: 1371.7743
Epoch 8/500
294/294 [==============================] - 1s 2ms/step - loss: 552.3179 - val_loss: 1336.9277
Epoch 9/500
294/294 [==============================] - 1s 2ms/step - loss: 542.4810 - val_loss: 1310.4281
Epoch 10/500
294/294 [=======================

Epoch 155/500
294/294 [==============================] - 1s 2ms/step - loss: 414.3597 - val_loss: 683.8323
Epoch 156/500
294/294 [==============================] - 1s 2ms/step - loss: 416.0023 - val_loss: 683.0367
Epoch 157/500
294/294 [==============================] - 1s 2ms/step - loss: 415.5233 - val_loss: 683.4517
Epoch 158/500
294/294 [==============================] - 1s 2ms/step - loss: 414.8613 - val_loss: 681.1290
Epoch 159/500
294/294 [==============================] - 1s 2ms/step - loss: 414.7094 - val_loss: 682.0212
Epoch 160/500
294/294 [==============================] - 1s 2ms/step - loss: 413.2574 - val_loss: 680.0659
Epoch 161/500
294/294 [==============================] - 1s 2ms/step - loss: 414.3063 - val_loss: 680.3361
Epoch 162/500
294/294 [==============================] - 1s 2ms/step - loss: 413.4843 - val_loss: 677.7864
Epoch 163/500
294/294 [==============================] - 1s 2ms/step - loss: 412.5424 - val_loss: 679.1146
Epoch 164/500
294/294 [==============

294/294 [==============================] - 1s 2ms/step - loss: 411.9855 - val_loss: 674.3026
Epoch 308/500
294/294 [==============================] - 1s 2ms/step - loss: 411.7653 - val_loss: 674.2352
Epoch 309/500
294/294 [==============================] - 1s 2ms/step - loss: 411.7701 - val_loss: 671.0092
Epoch 310/500
294/294 [==============================] - 1s 2ms/step - loss: 411.3572 - val_loss: 672.2383
Epoch 311/500
294/294 [==============================] - 1s 2ms/step - loss: 411.8103 - val_loss: 673.4547
Epoch 312/500
294/294 [==============================] - 1s 2ms/step - loss: 411.7061 - val_loss: 673.7936
Epoch 313/500
294/294 [==============================] - 1s 2ms/step - loss: 411.8069 - val_loss: 672.1429
Epoch 314/500
294/294 [==============================] - 1s 2ms/step - loss: 411.8121 - val_loss: 673.1257
Epoch 315/500
294/294 [==============================] - 1s 2ms/step - loss: 411.6510 - val_loss: 671.6476
Epoch 316/500
294/294 [============================

294/294 [==============================] - 1s 2ms/step - loss: 411.3270 - val_loss: 671.5995
Epoch 460/500
294/294 [==============================] - 1s 2ms/step - loss: 411.3363 - val_loss: 671.3859
Epoch 461/500
294/294 [==============================] - 1s 2ms/step - loss: 411.1851 - val_loss: 670.7201
Epoch 462/500
294/294 [==============================] - 1s 2ms/step - loss: 411.1656 - val_loss: 671.7360
Epoch 463/500
294/294 [==============================] - 1s 2ms/step - loss: 410.9930 - val_loss: 671.3276
Epoch 464/500
294/294 [==============================] - 1s 2ms/step - loss: 411.1336 - val_loss: 671.2465
Epoch 465/500
294/294 [==============================] - 1s 2ms/step - loss: 411.5304 - val_loss: 671.5427
Epoch 466/500
294/294 [==============================] - 1s 2ms/step - loss: 411.3780 - val_loss: 672.3859
Epoch 467/500
294/294 [==============================] - 1s 2ms/step - loss: 411.4908 - val_loss: 671.7764
Epoch 468/500
294/294 [============================

325/325 [==============================] - 1s 2ms/step - loss: 326.8660 - val_loss: 659.0198
Epoch 113/500
325/325 [==============================] - 1s 2ms/step - loss: 328.5532 - val_loss: 662.4733
Epoch 114/500
325/325 [==============================] - 1s 2ms/step - loss: 327.0027 - val_loss: 655.6168
Epoch 115/500
325/325 [==============================] - 1s 2ms/step - loss: 328.1409 - val_loss: 657.7720
Epoch 116/500
325/325 [==============================] - 1s 2ms/step - loss: 327.7002 - val_loss: 659.5773
Epoch 117/500
325/325 [==============================] - 1s 2ms/step - loss: 327.8760 - val_loss: 660.1966
Epoch 118/500
325/325 [==============================] - 1s 2ms/step - loss: 328.2015 - val_loss: 658.4451
Epoch 119/500
325/325 [==============================] - 1s 2ms/step - loss: 326.8663 - val_loss: 660.6731
Epoch 120/500
325/325 [==============================] - 1s 2ms/step - loss: 326.9445 - val_loss: 660.6561
Epoch 121/500
325/325 [============================

...

322/322 [==============================] - 1s 2ms/step - loss: 378.8141 - val_loss: 438.1534
Epoch 422/500
322/322 [==============================] - 1s 2ms/step - loss: 378.9638 - val_loss: 438.2116
Epoch 423/500
322/322 [==============================] - 1s 2ms/step - loss: 378.7938 - val_loss: 438.1377
Epoch 424/500
322/322 [==============================] - 1s 2ms/step - loss: 378.8933 - val_loss: 438.3696
Epoch 425/500
322/322 [==============================] - 1s 2ms/step - loss: 379.0802 - val_loss: 438.5181
Epoch 426/500
322/322 [==============================] - 1s 2ms/step - loss: 379.2188 - val_loss: 438.4545
Epoch 427/500
322/322 [==============================] - 1s 2ms/step - loss: 379.1091 - val_loss: 438.2807
Epoch 428/500
322/322 [==============================] - 1s 2ms/step - loss: 378.8508 - val_loss: 438.2657
Epoch 429/500
322/322 [==============================] - 1s 2ms/step - loss: 378.7106 - val_loss: 438.1046
Epoch 430/500
322/322 [============================

Epoch 74/500
320/320 [==============================] - 1s 3ms/step - loss: 393.8958 - val_loss: 395.5452
Epoch 75/500
320/320 [==============================] - 1s 2ms/step - loss: 390.3939 - val_loss: 393.8868
Epoch 76/500
320/320 [==============================] - 1s 2ms/step - loss: 392.6457 - val_loss: 394.2688
Epoch 77/500
320/320 [==============================] - 1s 2ms/step - loss: 392.3430 - val_loss: 395.1933
Epoch 78/500
320/320 [==============================] - 1s 2ms/step - loss: 391.7446 - val_loss: 394.8763
Epoch 79/500
320/320 [==============================] - 1s 2ms/step - loss: 391.7748 - val_loss: 395.0140
Epoch 80/500
320/320 [==============================] - 1s 2ms/step - loss: 390.2310 - val_loss: 394.3670
Epoch 81/500
320/320 [==============================] - 1s 2ms/step - loss: 391.1469 - val_loss: 394.6978
Epoch 82/500
320/320 [==============================] - 1s 2ms/step - loss: 391.1270 - val_loss: 394.2778
Epoch 83/500
320/320 [========================

Epoch 151/500
320/320 [==============================] - 1s 2ms/step - loss: 379.6570 - val_loss: 391.4664
Epoch 152/500
320/320 [==============================] - 1s 2ms/step - loss: 379.0587 - val_loss: 391.1423
Epoch 153/500
320/320 [==============================] - 1s 2ms/step - loss: 379.9169 - val_loss: 391.4691
Epoch 154/500
320/320 [==============================] - 1s 2ms/step - loss: 378.7216 - val_loss: 391.1935
Epoch 155/500
320/320 [==============================] - 1s 2ms/step - loss: 379.6640 - val_loss: 391.3628
Epoch 156/500
320/320 [==============================] - 1s 2ms/step - loss: 378.6354 - val_loss: 390.9678
Epoch 157/500
320/320 [==============================] - 1s 2ms/step - loss: 380.0693 - val_loss: 391.3769
Epoch 158/500
320/320 [==============================] - 1s 2ms/step - loss: 379.6087 - val_loss: 391.2605
Epoch 159/500
320/320 [==============================] - 1s 2ms/step - loss: 379.1469 - val_loss: 391.0364
Epoch 160/500
320/320 [==============

320/320 [==============================] - 1s 2ms/step - loss: 378.5363 - val_loss: 390.5869
Epoch 304/500
320/320 [==============================] - 1s 2ms/step - loss: 378.2241 - val_loss: 390.5276
Epoch 305/500
320/320 [==============================] - 1s 2ms/step - loss: 378.6228 - val_loss: 390.5765
Epoch 306/500
320/320 [==============================] - 1s 2ms/step - loss: 378.4324 - val_loss: 390.7126
Epoch 307/500
320/320 [==============================] - 1s 2ms/step - loss: 378.4937 - val_loss: 390.7417
Epoch 308/500
320/320 [==============================] - 1s 2ms/step - loss: 378.2973 - val_loss: 390.7133
Epoch 309/500
320/320 [==============================] - 1s 2ms/step - loss: 378.6314 - val_loss: 390.8176
Epoch 310/500
320/320 [==============================] - 1s 2ms/step - loss: 378.6402 - val_loss: 390.6840
Epoch 311/500
320/320 [==============================] - 1s 2ms/step - loss: 378.5382 - val_loss: 390.7594
Epoch 312/500
320/320 [============================

320/320 [==============================] - 1s 2ms/step - loss: 378.4946 - val_loss: 390.6707
Epoch 456/500
320/320 [==============================] - 1s 2ms/step - loss: 378.4601 - val_loss: 390.7111
Epoch 457/500
320/320 [==============================] - 1s 2ms/step - loss: 378.5325 - val_loss: 390.6758
Epoch 458/500
320/320 [==============================] - 1s 2ms/step - loss: 378.4430 - val_loss: 390.6401
Epoch 459/500
320/320 [==============================] - 1s 2ms/step - loss: 378.4536 - val_loss: 390.6683
Epoch 460/500
320/320 [==============================] - 1s 2ms/step - loss: 378.5484 - val_loss: 390.7280
Epoch 461/500
320/320 [==============================] - 1s 2ms/step - loss: 378.5078 - val_loss: 390.6719
Epoch 462/500
320/320 [==============================] - 1s 2ms/step - loss: 378.4351 - val_loss: 390.6586
Epoch 463/500
320/320 [==============================] - 1s 2ms/step - loss: 378.5300 - val_loss: 390.6827
Epoch 464/500
320/320 [============================

In [21]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

20.231669123803528

In [22]:
rmse_list

[11.103613447191488,
 28.319754563606924,
 15.895465197375211,
 21.529010545581556,
 12.659541111924504,
 25.807042660939345,
 20.138937015807194,
 13.036610340648341,
 26.02534203093943,
 27.80137432402127]